In [ ]:
%matplotlib

import math 
import pywt 
import numpy as np
import os
import csv
import fnmatch
import datetime
import re 
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats

In [ ]:
TimeDwellOrig = 800
TimeFixation = 300

In [ ]:
def filterBlinksNew(pupilData, timeListComplete):
    # filter any blinks and nan values lasting around 250ms (on average)
    # http://faculty.washington.edu/chudler/facts.html
   
    
   # Blink detection    
    missingPupilData = dict()
    missingPupilData['Start'] = list()
    missingPupilData['End'] = list()
    missingPupilData['Duration'] = list()
    missingPupilData['DurationExtra'] = list()
    
    pupilData_filter = list()
    time_filter = list()
    index_blinkEnd = list()
    
    missingBeginning = 0
    missingEnd = 0

    # First, find the missing values
    pupilInd = -1
    for pupilD in pupilData:
        
        pupilInd = pupilInd + 1
        #if pupilInd>772 and pupilInd<940:
        #    print(pupilInd, pupilD)
            
        # check if val is nan
        if np.isnan(pupilD):
            # check if first element of list
            if pupilInd == 0:
                # first value is nan
                missingPupilData['Start'].append(pupilInd)
                #print('Start', missingPupilData['Start'][-1])
                
            # check if previous val was nan
            elif np.isnan(pupilData[pupilInd-1]):
                
                # check if val is not last element
                if pupilInd < len(pupilData)-1:
                    # check if next val is not nan
                    if not np.isnan(pupilData[pupilInd+1]):
                        # note the index for end of blink
                        missingPupilData['End'].append(pupilInd)
                        #print('End', missingPupilData['End'][-1])
                else:
                    # last value is nan
                    missingEnd = 1
                    missingPupilData['End'].append(pupilInd)
                    #print('End', missingPupilData['End'][-1])
                    
            # if new nan, note the index
            else:
                missingPupilData['Start'].append(pupilInd)
                #print('Start', missingPupilData['Start'][-1])
                
                # for single nan values
                # check if val is not last element
                if pupilInd < len(pupilData)-1:
                    # check if next val is not nan
                    if not np.isnan(pupilData[pupilInd+1]):
                        # note the index for end of blink
                        missingPupilData['End'].append(pupilInd)
                        #print('End', missingPupilData['End'][-1])
                else:
                    # last value is nan
                    missingPupilData['End'].append(pupilInd)
                    #print('End', missingPupilData['End'][-1])
            
    
    
    # Second, create a list that does not contain missing values or 200ms before and after that
    
     # blink is every nan value in the range of 100-400ms 
    # 200 ms (18 samples) before and after the blink will also be removed
    addedBlinkSamples = 18    
    blinkDurationStart = 0
    blinkDurationEnd = 0
    
    iCurrent = len(missingPupilData['Start'])
    endCurrent = iCurrent - 1
    isBlink = 0
    pupilData_filter = list(pupilData)
    time_filter = list(timeListComplete)
    
    index_blinkEndExtra = list() # list to document if 2 blinks joined 
    
    while iCurrent > 0:
        
        iCurrent = iCurrent - 1
        
        #print('Current', missingPupilData['Start'][iCurrent], missingPupilData['End'][iCurrent])
        
        # check if it is really a blink, and remove if it is not
        # if blink duration is less than 100ms, the reason for missing data is something else
        timeStart0 = time_filter[missingPupilData['Start'][iCurrent]].partition('+')
        timeStart = (datetime.datetime.strptime(re.sub('[:.T]','-',timeStart0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
        timeEnd0 = time_filter[missingPupilData['End'][iCurrent]].partition('+')
        timeEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',timeEnd0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
        time1 = (timeEnd - timeStart).total_seconds()
        #print(time1)
        if time1 < 0.1 or time1 > 2:
            isNotBlink = 1
        else:
            isNotBlink = 0
            
        # Remove if it is a blink
        if isNotBlink:
            del pupilData_filter[missingPupilData['Start'][iCurrent]:missingPupilData['End'][iCurrent]+1]
            del time_filter[missingPupilData['Start'][iCurrent]:missingPupilData['End'][iCurrent]+1]
            
            # length of non-blink
            lengthBlink = missingPupilData['End'][iCurrent] - missingPupilData['Start'][iCurrent] + 1
            
            # all blinks after this non-blink need to be reduced in indexes
            #print(len(missingPupilData['Start'])-1)
            
            for iChange in range(iCurrent-1, len(missingPupilData['Start'])-1):
                #print('i', iChange)
                if iChange == iCurrent-1:
                    
                    #print('Deleted', missingPupilData['Start'][iChange+1], missingPupilData['End'][iChange+1])
                    #print(lengthBlink)
                    del missingPupilData['Start'][iChange+1]
                    del missingPupilData['End'][iChange+1]
                else:
                    
                    #print('Old', missingPupilData['Start'][iChange], missingPupilData['End'][iChange])
                    missingPupilData['Start'][iChange] = missingPupilData['Start'][iChange] - lengthBlink
                    missingPupilData['End'][iChange] = missingPupilData['End'][iChange] - lengthBlink
                    #print('New', missingPupilData['Start'][iChange], missingPupilData['End'][iChange])
                    
    #print(len(missingPupilData['Start']))
    
    previousBlinkEnd = 0
    nextBlinkEnd = 0
    iCurrent = len(missingPupilData['Start'])
    
#     for index, (value1, value2) in enumerate(zip(missingPupilData['Start'], missingPupilData['End'])):
#         print(index, value1, value2)
#     for i in enumerate(pupilData_filter):
#         print(i)
    
    while iCurrent > 0:
        
        iCurrent = iCurrent - 1
        #print(iCurrent)
        iNow = iCurrent # iCurrent is modified in advance later (possibly), which will disturb the current data index
        #print(missingPupilData['Start'][iNow], missingPupilData['End'][iNow])
        
#         if iNow > 1 :
#             print('BEFORE!!!!!!!!!!!!!!!!!')
#             print(pupilData_filter[missingPupilData['Start'][iNow]:-1])
            
        # to make sure that blink removal data does not go into the previous data
        if iCurrent == 0:
            previousBlinkEnd = 0
        else:
            previousBlinkEnd = missingPupilData['Start'][iCurrent-1]

        # to make sure blink does not go into the next one
        if iCurrent == len(missingPupilData['Start'])-1:
            #print(iCurrent)
            nextBlinkEnd = len(pupilData_filter)
            #print(nextBlinkEnd)
        else:
            nextBlinkEnd = missingPupilData['Start'][iCurrent+1]
            
        
        
        if missingPupilData['End'][iCurrent] + addedBlinkSamples > nextBlinkEnd:
            #LOOK INTO THIS, NEXT BLINK IS REMOVED BUT ITS EXTRA 200MS IS NOT!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            #print('After', missingPupilData['End'][iCurrent], addedBlinkSamples, '>>>>>', nextBlinkEnd)
            blinkRemoveAfter = abs(missingPupilData['End'][iCurrent] - nextBlinkEnd)
        else:
            blinkRemoveAfter = addedBlinkSamples
        
        if missingPupilData['Start'][iCurrent] - addedBlinkSamples < previousBlinkEnd:
            #print('Before', missingPupilData['Start'][iCurrent], addedBlinkSamples, '<<<<<<', previousBlinkEnd)
            blinkRemoveBefore = abs(missingPupilData['Start'][iCurrent] - previousBlinkEnd)
            
            # Add the extra duration to list
            if iCurrent > 0:
                
                timeStart0 = time_filter[missingPupilData['Start'][iCurrent-1]].partition('+')
                timeStart = (datetime.datetime.strptime(re.sub('[:.T]','-',timeStart0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
                timeEnd0 = time_filter[missingPupilData['End'][iCurrent-1]].partition('+')
                timeEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',timeEnd0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
                
                time1 = (timeEnd - timeStart).total_seconds()
                #print(timeEnd, timeStart, time1)

                missingPupilData['DurationExtra'].append(time1)
#             if iCurrent > 1:
#                 print(missingPupilData['Start'][iCurrent-1], 'also removed')
#             # After next blink is also removed, 
            iCurrent = iCurrent - 1
            index_blinkEndExtra.append(2)
            
                
        else:
            blinkRemoveBefore = addedBlinkSamples
            index_blinkEndExtra.append(1)
            missingPupilData['DurationExtra'].append(0)
            
#         print(missingPupilData['Start'][iNow]-blinkRemoveBefore)
        
        timeStart0 = time_filter[missingPupilData['Start'][iNow]].partition('+')
        timeStart = (datetime.datetime.strptime(re.sub('[:.T]','-',timeStart0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
        #print(timeStart, pupilData_filter[missingPupilData['Start'][iNow]])     
        timeEnd0 = time_filter[missingPupilData['End'][iNow]].partition('+')
        timeEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',timeEnd0[0][:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
        #print(timeEnd) 
        time2 = (timeEnd - timeStart).total_seconds()
        #print(time2)
        missingPupilData['Duration'].append(time2)
        del pupilData_filter[missingPupilData['Start'][iNow]-blinkRemoveBefore:missingPupilData['End'][iNow]+blinkRemoveAfter]
        del time_filter[missingPupilData['Start'][iNow]-blinkRemoveBefore:missingPupilData['End'][iNow]+blinkRemoveAfter]
        
        index_blinkEnd.append(missingPupilData['Start'][iNow] - blinkRemoveBefore - 1)
    #         if iNow > 1 :
#             print('AFTER!!!!!!!!!!!!')
#             if np.nan in pupilData_filter[missingPupilData['Start'][iNow]:-1]:
#                 for i,j in enumerate(pupilData_filter[missingPupilData['Start'][iNow]:-1]):
#                     print(i+missingPupilData['End'][iNow]+blinkRemoveAfter,j)
        
    
    if np.nan in pupilData_filter:
        for i in enumerate(pupilData_filter):
            print(i)
     
    index_blinkEnd = list(reversed(index_blinkEnd))
    
    return pupilData_filter, time_filter, index_blinkEnd, index_blinkEndExtra, missingPupilData

In [ ]:
def ComputeDwellTime(userKeys):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        if key[1] == 'IncreaseTimeDwell':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseTimeDwell':
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [ ]:
def UserLookedAtNewTrial(UserLooksAtKey):
    # Create list of list (made of epochs) composed of times when user looked at NextPhrase
    
    epoch = list()   
    epochList = list()
    
    nRow = -1
    firstNextPhrase = 0
    flagStart = 0
    firstTrial = 0
    
    # Find unique looks at new trial, by checking if progress pct is greater than previous one and store first look in a list
    
    progressNextPhrase = 0 
    
    for row in UserLooksAtKey:
        nRow = nRow + 1
        
        # first add data for first trial, before first next phrase is reached
        if firstTrial == 0:
            firstTrial = 1
            epochList.append([row, 0])
            ##print(epochList)
            continue
            
        if row[1] == 'NextPhrase':
            
            # first eliminate an ongoing look
            if UserLooksAtKey[nRow-1][1] == 'NextPhrase':
                continue
            else:
                # normally for new looks
                
                # if it is a continued look, progressNextPhrase should not be 0
                ##print('Continued ???', progressNextPhrase, UserLooksAtKey[nRow])
                
                if progressNextPhrase > float(UserLooksAtKey[nRow][2]):
                    
                    # save it as 0, if not a continued look
                    ##print('progress is now 0')
                    progressNextPhrase = 0
#                 else:
#                     if progressNextPhrase == float(UserLooksAtKey[nRow][2]):
#                         #print('ITS EQUAL!!!')
                    
                
            # now check if this look is going to be successfully selected:
            nextPhrase_selected = 0
            rowCurrent = nRow-1
            
            while nextPhrase_selected < 1:
                rowCurrent = rowCurrent + 1
                if UserLooksAtKey[rowCurrent][1] == 'NextPhrase':
                    if float(UserLooksAtKey[rowCurrent][2]) == 1:
                        ##print(UserLooksAtKey[rowCurrent])
                        nextPhrase_selected = 1
                        rowCurrentEnd = rowCurrent
                        
                        
                        if progressNextPhrase != 0:
                            epochList[-1].insert(2, UserLooksAtKey[rowPreviousEnd])
                            epochList.append([UserLooksAtKey[rowCurrentStart], UserLooksAtKey[rowCurrentEnd]])
                        else:
                            epochList[-1].insert(2, UserLooksAtKey[nRow-1])
                            epochList.append([UserLooksAtKey[nRow], UserLooksAtKey[rowCurrentEnd]])
                    else:
                        continue
                else:
                    # if look at next phrase is stopped, store the previous look progress, in case it is continued later
                    progressNextPhrase = float(UserLooksAtKey[rowCurrent-1][2])
                    rowCurrentStart = nRow
                    rowPreviousEnd = nRow - 1
                    break
                    
                     
                    
                    
    # insert last element before quiting - 
    # keys looked at list
    KeysLookedAtFromUserKeys = [key[1] for key in UserLooksAtKey]
    
    # find key before quit key
    nBegin = 300
    foundQuitKey = 0
    indBeforeQuit = 0
    while foundQuitKey < 1:
        if KeysLookedAtFromUserKeys[-nBegin] == 'Quit':
            # look in direction of previous keys
            # if previous key is not quit, you can stop
            if KeysLookedAtFromUserKeys[-nBegin-1] != 'Quit':
                foundQuitKey = 1
                indBeforeQuit = -nBegin - 1
                break
            else:
                nBegin = nBegin + 1
        else:
            # current key is not quit, start looking in the other direction
            if KeysLookedAtFromUserKeys[-nBegin + 1] == 'Quit':
                foundQuitKey = 1
                indBeforeQuit = -nBegin
                break
            else:
                nBegin = nBegin - 1
    
    epochList[-1].insert(2, UserLooksAtKey[indBeforeQuit])
    
    #for i in epochList:
    #    #print(i)
    
    return epochList

In [ ]:
# function to convert list of date and time into datetime format list

def timeConversion(timeStrList):
    timeListNew = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeListNew.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeListNew

In [ ]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [ ]:
def modmax(d):
    # modulus maxima detection
    
    # compute signal modulus
    m = [0.0]*len(d)
    for i in range(0, len(d)):
        m[i] = math.fabs(d[i])
    
    # if value is larger than both neighbours , and strictly
    # larger than either , then it is a local maximum
    t = [0.0]*len(d)
    for i in range(0, len(d)):
        ll = m[i-1] if i >= 1 else m[i]
        oo = m[i]
        rr = m[i+1] if i < len(d)-2 else m[i]
        if (ll <= oo and oo >= rr) and (ll < oo or oo > rr):
            # compute magnitude
            t[i] = math.sqrt(d[i]**2)
        else:
            t[i] = 0.0
    ##print(len(t))
    return t

In [ ]:
def ipa(d):
    # compute ipa value of pupil diameter
    IPA = list()
    ##print(len(d.pupildata.values))
    # obtain 2-level DWT of pupil diameter signal d
    
    try:
        (cA2,cD2,cD1) = pywt.wavedec(d.pupildata.values,'sym16','per',level=2)
    except ValueError:
        ##print(ValueError)
        #print('value error in wavedec')
        return
    
#     (cA3,cD3,cD2,cD1) = pywt.wavedec(d.pupildata.values,'sym16','per',level=3)
    
    # get signal duration (in seconds)
    tt = ((d.timestamp.values[-1] - d.timestamp.values[0]).item())/1000000000
    
    ##print(tt)
    
## TWO LEVEL WAVELET DECOMPOSITION
    # normalize by 1=2j , j = 2 for 2-level DWT
    cA2[:] = [x / math.sqrt(4.0) for x in cA2]
    cD1[:] = [x / math.sqrt(2.0) for x in cD1]
    cD2[:] = [x / math.sqrt(4.0) for x in cD2]
    
    # detect modulus maxima , see Listing 2
    cD2m = modmax(cD2)
    ##print(len(cD2m))
    
    # threshold using universal threshold l_univ = s*sqrt(2logn)
    # where s is the standard deviation of the noise
    luniv = np.std(cD2m) * math.sqrt(2.0*np.log2(len(cD2m)))
    cD2t = pywt.threshold(cD2m ,luniv,mode="hard")
    
    # compute IPA
    ctr = 0
    for i in range(0, len(cD2t)):
        if math.fabs(cD2t[i]) > 0: ctr += 1
        #IPA = float(ctr)/tt
        # maybe each pupil data has an IPA?
    IPA = (float(ctr)/tt)
    
    return IPA, cD2m, cD2t, cD2, cD1, cA2, tt

# # THREE LEVEL WAVELET DECOMPOSITION

# # normalize by 1=2j , j = 2 for 2-level DWT
#     cA3[:] = [x / math.sqrt(8.0) for x in cA3]
#     cD3[:] = [x / math.sqrt(4.0) for x in cD3]
#     cD1[:] = [x / math.sqrt(4.0) for x in cD1]
#     cD2[:] = [x / math.sqrt(8.0) for x in cD2]
    
#     # detect modulus maxima , see Listing 2
#     cD3m = modmax(cD3)
    
#     # threshold using universal threshold l_univ = s*sqrt(2logn)
#     # where s is the standard deviation of the noise
#     luniv = np.std(cD3m) * math.sqrt(2.0*np.log2(len(cD3m)))
#     cD3t = pywt.threshold(cD3m ,luniv,mode="hard")
    
#     # compute IPA
#     ctr = 0
#     for i in range(0, len(cD3t)):
#         if math.fabs(cD3t[i]) > 0: ctr += 1
#         #IPA = float(ctr)/tt
#         # maybe each pupil data has an IPA?
#     IPA = (float(ctr)/tt)
    
#     return IPA, cD3m, cD3t, cD3, cD2, cD1, tt


In [ ]:
def evaluateTrials(trialEpoch, scratchPadList, phraseList, gazeLog, timeGazeLog, ipaDict, index_blinkEnd, index_BlinkEndExtra, missingData, subjName):
    

    # customise for one subj
    if subjName == 'mm_MS' or subjName == 'sh_MS':
        trialEpoch.remove(trialEpoch[-1])
     
    
        
    phraseScratchedList = list()
    phraseToBeScratchedList = list()
    timeUserLooked = list()
   
    trialNum = 0
    
    phraseScratchedInd = 0
    phraseToBeScratchedInd = 0
    
    timeStrScratch = [item1[0] for item1 in scratchPadList]
    timeStrToBeScracthed = [item2[0] for item2 in phraseList]
    
    timeScratchList = timeConversion(timeStrScratch)
    timeToBeScratchedList = timeConversion(timeStrToBeScracthed)
    
    
    rowTimeList = -1
    for timeDuration in trialEpoch:
        
        rowTimeList = rowTimeList + 1
        
        # Extract epoch data from gazelog
        
        time1, t1, t2 = timeDuration[0][0].partition('+')
        timeTrialStart = (datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
        # find the actual start of the fixation
        timeScratchPadActive = timeTrialStart - datetime.timedelta(milliseconds=float(timeDuration[0][3])) - datetime.timedelta(milliseconds=TimeFixation)
        timeGazeLogStart, GazeLogStartInd = nearestTimePoint(timeGazeLog, timeScratchPadActive)
        
        time2, t1, t2 = timeDuration[-1][0].partition('+')
        timeTrialEnd = (datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
        timeGazeLogEnd, GazeLogEndInd = nearestTimePoint(timeGazeLog, timeTrialEnd)
        
        #print(timeGazeLogEnd, timeGazeLogStart)
        if timeGazeLogEnd - timeGazeLogStart < datetime.timedelta(seconds = 10):
            ##print(timeGazeLogEnd - timeGazeLogStart)
            ##print('Too short to be a trial')
            ##print(userKeys[userKeys.index(trialTime[rowTimeList][0]):userKeys.index(trialTime[rowTimeList][1])])
            ##print('NEXT!!!!!!!!')
            continue
        #else:
        #    #print(timeGazeLogEnd - timeGazeLogStart)
            ##print('NEXT!!!!!!!!')
        
        ##print('trialtime', timeTrialStart, timeTrialEnd)
        
        ##print(GazeLogStartInd)
        
        # Find number of blinks in the trial
        blinkCountStart = [i for i,v in enumerate(index_blinkEnd) if GazeLogStartInd<v]
#         #print('gaze start', GazeLogStartInd)
        if len(blinkCountStart) < 1:
            blinkCountStartInd = 0
        else:
            blinkCountStartInd = blinkCountStart[0]
    
#         print('trial start-end', GazeLogStartInd, GazeLogEndInd)
#         print('Blink ends', index_blinkEnd)
        
#         print('blink start thingy', blinkCountStartInd)  
#         print('whole start^^', blinkCountStart)
        
        blinkCountEnd = [i for i,v in enumerate(index_blinkEnd) if GazeLogEndInd>v]
        
        
        #print('gaze end', GazeLogEndInd)
        if len(blinkCountEnd) < 1:
            blinkCountEndInd = 0
        else:
            blinkCountEndInd = blinkCountEnd[-1]
        
        
#         print('blink end', blinkCountEndInd)
#         print('whole End', blinkCountEnd)
        
        blinkFreqTrial = blinkCountEndInd-blinkCountStartInd+1
        
#         print('Initial Frequency', blinkFreqTrial)
#         print('Extraaaaas', index_BlinkEndExtra)
        for blink in range(blinkCountStartInd, blinkCountEndInd+1):
            if index_BlinkEndExtra[blink] > 1:
                blinkFreqTrial = blinkFreqTrial + 1
        
        # Find blink duration from missingData
        # first create list of index of blinks in this trial
        blinkIndexList = list(range(blinkCountStartInd, blinkCountEndInd+1))
        
        blinkDurationList = [missingData['Duration'][i] for i in blinkIndexList]
        blinkExtraDurationList = [missingData['DurationExtra'][i] for i in blinkIndexList]
        

        blinkExtraDurationFinal = [blink for blink in blinkExtraDurationList if blink>0]
#        print(blinkExtraDurationFinal)
        #if len(blinkExtraDurationFinal) > 0:
        #    print(blinkExtraDurationFinal)
        blinkDurationFinal = blinkDurationList + blinkExtraDurationFinal
        if len(blinkDurationFinal) > 0:
            blinkDurationAverage = np.mean(blinkDurationFinal)
        else:
            blinkDurationAverage = 0
            
        #print('Final frequency', blinkFreqTrial)
        
        # Find the stimPhrases and the phraseTyped
        
        timeOfPhraseScratchedNow, phraseScratchedInd = nearestTimePoint(timeScratchList, timeTrialEnd)
        timeOfPhraseToBeScratchedNow, phraseToBeScratchedInd = nearestTimePoint(timeToBeScratchedList, timeTrialEnd)
        
        ##print('phrasetime', timeOfPhraseToBeScratchedNow, timeOfPhraseScratchedNow)
        
        if phraseScratchedInd < 0: # sometimes there are random things looked at, in userkeys but not typed
            phraseScratched = ''
        elif scratchPadList[phraseScratchedInd+1][1] != '': # sometimes scratchpad is updated a slight ms after userkeys
            phraseScratched = scratchPadList[phraseScratchedInd+1][1]
            ##print('weird timing:    ', phraseScratched)
        else:
            phraseScratched = scratchPadList[phraseScratchedInd][1]
        
        phraseToBeScratched = phraseList[phraseToBeScratchedInd][1]
        
        
        if 'THE EXPERIMENT IS NOW DONE' in phraseToBeScratched:
            ##print('exp done')
            continue
        
        ##print(phraseToBeScratched, '   AND   ', phraseScratched)
        
        # find pupil size for the trial
        pupilLog = gazeLog[GazeLogStartInd:GazeLogEndInd]
        
        # compute ipa for the trial - after the phrases, to not compute if expt is done
        pupilLog_filter_wTime_Tuple = list(zip(time_filterFormatted, pupilLog))
        pupilLog_pd =  pd.DataFrame(pupilLog_filter_wTime_Tuple, columns=['timestamp','pupildata'])
        ipaVal, coeff_modmax, coeff_hard, coeff_D2, coeff_D1, coeff_A, timePeriodTrial = ipa(pupilLog_pd)
        
        
        # are they the same? yes/no -> save in list with the time
        
        # if the phrase to by typed contains numbers, they are not typed
        
        if not any(char.isdigit() for char in phraseToBeScratched):
            # no number is present
            if phraseToBeScratched.lower() in phraseScratched.lower():
                # it is correct
                ipaDict['C'].append(ipaVal)
                ipaDict['C_blinkRate'].append((blinkFreqTrial*60)/timePeriodTrial)
                ipaDict['C_blinkDuration'].append(blinkDurationAverage)
#                 print('C')
#                 print((blinkFreqTrial*60)/timePeriodTrial)
            else:
                ipaDict['INC'].append(ipaVal)
                ipaDict['INC_blinkRate'].append((blinkFreqTrial*60)/timePeriodTrial)
                ipaDict['INC_blinkDuration'].append(blinkDurationAverage)
#                 print('INC')
#                 print((blinkFreqTrial*60)/timePeriodTrial)
        else:
            ##print(phraseToBeScratched)
            if phraseScratched.lower() in phraseToBeScratched.lower():
                # it is correct
                ipaDict['C'].append(ipaVal)
                ipaDict['C_blinkRate'].append((blinkFreqTrial*60)/timePeriodTrial)
                ipaDict['C_blinkDuration'].append(blinkDurationAverage)
#                 print('C')
#                 print((blinkFreqTrial*60)/timePeriodTrial)
            else:
                ipaDict['INC'].append(ipaVal)
                ipaDict['INC_blinkRate'].append((blinkFreqTrial*60)/timePeriodTrial)
                ipaDict['INC_blinkDuration'].append(blinkDurationAverage)
#                 print('INC')
#                 print((blinkFreqTrial*60)/timePeriodTrial)

        trialNum = trialNum + 1
        #print(blinkFreqTrial, timePeriodTrial)
    ##print(trialNum)
    return ipaDict
    
    

In [ ]:
subjName = r'C:\DTU\Data\201805_HealthnRehab\TypingData'
j = 0
flagFirstSubj = 0
blinkFreq = list()
ipaDict = dict()
ipaDict['C'] = list()
ipaDict['INC'] = list()
ipaDict['C_blinkRate'] = list()
ipaDict['INC_blinkRate'] = list()
ipaDict['C_blinkDuration'] = list()
ipaDict['INC_blinkDuration'] = list()

for root, dirs, subfolder in os.walk(subjName):
    LetterLookedAtList = list()
    LetterLookedAt = list()
    
    if not dirs:
        
        if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'tb' in root or 'joha' in root:
            continue
            
        userKeys = None
        scratchPad = None
        gazeLog = None
        stimPhrase = None
        
        for file in subfolder:
            if fnmatch.fnmatch(file, 'user_look*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
            elif fnmatch.fnmatch(file, 'ScratchPad*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8')
                    readerScratchPad = csv.reader(fScratchPad)
                    scratchPad = list(readerScratchPad)  
                    scratchPad.remove(scratchPad[0])
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the user looks at log file')
            elif fnmatch.fnmatch(file, 'PhraseLog*'):
                try:
                    fStimPhrase = open(root + '\\' + file, encoding='utf-8')
                    readerStimPhrase = csv.reader(fStimPhrase)
                    stimPhrase = list(readerStimPhrase)
                    stimPhrase.remove(stimPhrase[0])
                except:
                    if fStimPhrase is not None:
                        fStimPhrase.close()
                    else:
                        print('error in opening the phrase log file')
            
            elif fnmatch.fnmatch(file, 'GazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
            else:
                continue
            
                # if all these lists exist
            if userKeys is None or scratchPad is None or stimPhrase is None or gazeLog is None:
                continue
            else:
                
                # compile subject names in a list
                a = re.compile('(?<=TypingData\\\May[0-9]{2}\\\)(.*)(?=\\\OptiKey)')
                subjName = a.findall(root)[0]
                typing_mechanism = subjName[-2:]
                print(subjName)
                
                # obtain right and left pupil data
                pupilLogL = [float(item4[29]) if 'Invalid' not in item4 else np.nan for item4 in gazeLog]
                pupilLogR = [float(item5[31]) if 'Invalid' not in item5 else np.nan for item5 in gazeLog]
                timeList = [item[0] for item in gazeLog]
                
                # filter the blinks
                pupilLogL_filter, time_filter, index_blinkEndL, index_blinkEndExtra, missingDataL = filterBlinksNew(pupilLogL, timeList)
                
                pupilLogR_filter, time_filter, index_blinkEndR, index_blinkEndExtra, missingDataR = filterBlinksNew(pupilLogR, timeList)
                
                time_filterFormatted = timeConversion(time_filter)
                pupilLog_avg = [(pupilLogL_filter[i] + pupilLogR_filter[i])/2 for i in range(0, len(pupilLogL_filter))]
                #pupilLog_avg = pupilLogL
                
                # Compute dwell time
                userKeysWithTime = ComputeDwellTime(userKeys)

                # call function to check when scratchpad is looked at and save it in a list
                trialTime = UserLookedAtNewTrial(userKeysWithTime)
                
                #for i in trialTime:
                #    ##print(i)
                    
                ipaDict = evaluateTrials(trialTime, scratchPad, stimPhrase, pupilLog_avg, time_filterFormatted, ipaDict, index_blinkEndL, index_blinkEndExtra, missingDataL, subjName)
                
                #for i in enumerate(time_filter):
                    ##print(i)
                
                

In [ ]:
print('Correct', np.mean(ipaDict['C']), np.std(ipaDict['C']))
print('Incorrect', np.mean(ipaDict['INC']), np.std(ipaDict['INC']))

print('Correct Blink rate', np.mean(ipaDict['C_blinkRate']), np.std(ipaDict['C_blinkRate']))
print('Incorrect Blink rate', np.mean(ipaDict['INC_blinkRate']), np.std(ipaDict['INC_blinkRate']))

print('Correct Blink duration', np.mean(ipaDict['C_blinkDuration']), np.std(ipaDict['C_blinkDuration']))
print('Incorrect Blink duration', np.mean(ipaDict['INC_blinkDuration']), np.std(ipaDict['INC_blinkDuration']))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1) 

ax.boxplot([ipaDict['C'], ipaDict['INC']], positions= [1, 2])
xTicks = ax.get_xticks()
#print(xTicks)
ax.set_xticklabels(['Correct', 'Incorrect'])

ax.set_title('Correct- mean: {0}, std: {1}                      Incorrect- mean:  {2},  std:  {3}'.format(np.round(np.mean(ipaDict['C']), 3), np.round(np.std(ipaDict['C']),3), np.round(np.mean(ipaDict['INC']),3), np.round(np.std(ipaDict['INC']),3)))
#ax.set_ylim([0, 0.5])

print(scipy.stats.ttest_ind(ipaDict['C'], ipaDict['INC']))
scipy.stats.ranksums(ipaDict['C'], ipaDict['INC'])


In [ ]:
# fig = plt.figure()
# ax = fig.add_subplot(1,1,1) 

# ax.hist(ipaDict['C_blinkRate'])
# ax.hist(ipaDict['INC_blinkRate'], alpha = 0.2)

fig = plt.figure()
ax = fig.add_subplot(1,1,1) 

ax.boxplot([ipaDict['C_blinkRate'], ipaDict['INC_blinkRate']], positions= [1, 2])
xTicks = ax.get_xticks()
#print(xTicks)
ax.set_xticklabels(['Correct blink rate', 'Incorrect blink rate'])

ax.set_title('Correct blink rate - mean: {0}, std: {1}          Incorrect blink rate- mean:  {2},  std:  {3}'.format(np.round(np.mean(ipaDict['C_blinkRate']), 3), np.round(np.std(ipaDict['C_blinkRate']),3), np.round(np.mean(ipaDict['INC_blinkRate']),3), np.round(np.std(ipaDict['INC_blinkRate']),3)))
#ax.set_ylim([-0.05, 0.4])
ax.grid('on')
print(scipy.stats.ttest_ind(ipaDict['C_blinkRate'], ipaDict['INC_blinkRate']))
scipy.stats.ranksums(ipaDict['C_blinkRate'], ipaDict['INC_blinkRate'])


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1) 

ax.boxplot([ipaDict['C_blinkDuration'], ipaDict['INC_blinkDuration']], positions= [1, 2])
xTicks = ax.get_xticks()
#print(xTicks)
ax.set_xticklabels(['Correct blink duration', 'Incorrect blink duration'])

ax.set_title('Correct blink duration - mean: {0}, std: {1}          Incorrect blink duration - mean:  {2},  std:  {3}'.format(np.round(np.mean(ipaDict['C_blinkDuration']), 3), np.round(np.std(ipaDict['C_blinkDuration']),3), np.round(np.mean(ipaDict['INC_blinkDuration']),3), np.round(np.std(ipaDict['INC_blinkDuration']),3)))
#ax.set_ylim([-0.05, 0.4])
ax.grid('on')
print(scipy.stats.ttest_ind(ipaDict['C_blinkDuration'], ipaDict['INC_blinkDuration']))
scipy.stats.ranksums(ipaDict['C_blinkDuration'], ipaDict['INC_blinkDuration'])

In [ ]:
fig = plt.figure()
axL = fig.add_subplot(2,1,1)
axR = fig.add_subplot(2,1,2)
ylim = [-0.02, 0.2]

xaxis = np.arange(0, len(coeff_left), 1/90)

axL.plot(coeff_left)
xTick = axL.get_xticks()
#axL.set_xticklabels(np.round(xTick*(1/90), 2))
axL.set_ylim(ylim)
index_blinkEndAdjust = [i/4 for i in index_blinkEnd]
#for xPoint in index_blinkEndAdjust:
#    axL.plot([xPoint, xPoint], ylim, '--', 'k', alpha = 0.3)

axR.plot(coeff_right)
axR.set_ylim(ylim)

In [ ]:
a = np.arange(0, len(coeff_left)*(4/90), 4/90)

In [ ]:
def plotCoeff_RL(coeffL, coeffR, title):
    
    fig = plt.figure()
    axL = fig.add_subplot(2,1,1)
    axR = fig.add_subplot(2,1,2)
                
    ylim = [np.min([min(coeffL), min(coeffR)])-0.02, np.max([max(coeffL), max(coeffR)]) + 0.02]

    xaxis = np.arange(0, len(coeffL), 1/90)

    axL.plot(coeffL)
    xTicks = axL.get_xticks()
    xTickLabels = [np.round(i*(4/90), 2) for i in xTicks[1:]]
    axL.set_xticks(xTicks[1:-1])
    axL.set_xticklabels(xTickLabels)
    axL.set_ylim(ylim)
    axL.set_title('subject:  {0},  coefficient for {1},  Left IPA  {2}'.format(subjName, title, np.round(ipa_left,4)))
    axL.set_xlabel('Time [in s]')
                
    axR.plot(coeffR)
    axR.set_ylim(ylim)
    axR.set_xticks(xTicks[1:-1])
    axR.set_xticklabels(xTickLabels)
    axR.set_title('Right IPA  {0}'.format(np.round(ipa_right, 4)))
    axR.set_xlabel('Time [in s]')
                
    plt.tight_layout()

In [ ]:
a = []
if len(a) < 1:
    #print('yo')

In [ ]:
a = [2, 3, 5]

In [ ]:
5*a

In [ ]:
a = dict()

a['b'] = list()
a['b'] = [1,6,2,7,3,8,3,7,1]

In [ ]:
print(len(a['b'][2:4]))

In [ ]:
del a['b'][2:4]

In [ ]:
a

In [ ]:
a['b']

In [ ]:
a = list(range(2,4))

In [ ]:
a